# DDColor ONNX to TensorRT Conversion

Simple notebook to convert DDColor ONNX model to TensorRT engine for faster GPU inference.

## 1. Configuration

In [1]:
# File paths
ONNX_MODEL_PATH = "./exported/model.onnx"
ENGINE_OUTPUT_PATH = "./exported/model.plan"

# Model settings
IMAGE_SIZE = 512  # Width and height
MIN_BATCH = 1
OPT_BATCH = 8   # Most common batch size
MAX_BATCH = 16

print(f"ONNX Model: {ONNX_MODEL_PATH}")
print(f"Output Engine: {ENGINE_OUTPUT_PATH}")
print(f"Batch sizes: min={MIN_BATCH}, opt={OPT_BATCH}, max={MAX_BATCH}")

ONNX Model: ./exported/model.onnx
Output Engine: ./exported/model.plan
Batch sizes: min=1, opt=8, max=16


## 2. Build TensorRT Command

In [2]:
# Build trtexec command
trtexec_cmd = f"""
trtexec \\
  --onnx={ONNX_MODEL_PATH} \\
  --saveEngine={ENGINE_OUTPUT_PATH} \\
  --fp16 \\
  --minShapes=input:{MIN_BATCH}x3x{IMAGE_SIZE}x{IMAGE_SIZE} \\
  --optShapes=input:{OPT_BATCH}x3x{IMAGE_SIZE}x{IMAGE_SIZE} \\
  --maxShapes=input:{MAX_BATCH}x3x{IMAGE_SIZE}x{IMAGE_SIZE}
""".strip()

print("TensorRT Conversion Command:")
print("=" * 50)
print(trtexec_cmd)
print("=" * 50)

TensorRT Conversion Command:
trtexec \
  --onnx=./exported/model.onnx \
  --saveEngine=./exported/model.plan \
  --fp16 \
  --minShapes=input:1x3x512x512 \
  --optShapes=input:8x3x512x512 \
  --maxShapes=input:16x3x512x512


## 3. Run Conversion

In [3]:
import subprocess
import os
import time

# Check if ONNX file exists
if not os.path.exists(ONNX_MODEL_PATH):
    print(f"⚠️  ONNX file not found at {ONNX_MODEL_PATH}")
    print("Please run the ONNX export notebook first.")
else:
    print("Starting TensorRT conversion...")
    print("This may take 2-5 minutes depending on your GPU.\n")
    
    start_time = time.time()
    
    # Run the conversion
    result = subprocess.run(
        trtexec_cmd.replace('\\', '').split(),
        capture_output=True,
        text=True
    )
    
    elapsed = time.time() - start_time
    
    if result.returncode == 0:
        print(f"✅ Conversion successful! (took {elapsed:.1f} seconds)")
        
        # Check file size
        if os.path.exists(ENGINE_OUTPUT_PATH):
            size_mb = os.path.getsize(ENGINE_OUTPUT_PATH) / (1024 * 1024)
            print(f"📁 Engine saved to: {ENGINE_OUTPUT_PATH}")
            print(f"📏 File size: {size_mb:.2f} MB")
    else:
        print(f"❌ Conversion failed!")
        print("Error output:")
        print(result.stderr[-1000:])  # Last 1000 chars of error

Starting TensorRT conversion...
This may take 2-5 minutes depending on your GPU.

✅ Conversion successful! (took 197.2 seconds)
📁 Engine saved to: ./exported/model.plan
📏 File size: 113.04 MB


## Summary

Your DDColor model has been converted to TensorRT format!

**Key points:**
- ✅ FP16 precision enabled for 2x speedup
- ✅ Dynamic batching from 1 to 16
- ✅ Optimized for 512x512 input images
- ✅ Ready for production deployment

The `.plan` file can now be used for fast GPU inference in your application.